<a href="https://colab.research.google.com/github/kurtlee1984/Modification_code/blob/main/YOLOv5_Pytorch_modificaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 將Colab與google_drive綁定
# 連接到指定帳號,並複製授權碼貼上

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 從 Github 下載 YOLOv5 的資料庫

!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

In [ ]:
# 安裝必要的開發套件與環境
!pip install -qr requirements.txt
import torch

# 顯示圖片的套件
from IPython.display import Image, clear_output

# 下載模型和資料集的套件
from utils.google_utils import gdrive_download

# 顯示出當前訓練的配備
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
# 下載資料集,並於 google_drive 取得壓縮檔的 id
# id 取得方法,到 google_drive 指定壓縮檔,滑鼠右鍵選擇取得連結,更改權限由限制 -> 知道連結的使用者,複製連結在筆記本上
# https://drive.google.com/file/d/這裡就是壓縮檔的id/view?usp=sharing

%cd /content
!gdown --id '1drP4tymbSW5UiHPP6W9Brx1Uud2Jag9j' -O dataset.zip

In [ ]:
# 解壓縮,注意壓縮檔名稱需與代碼一致
# 解壓縮完,於右邊選擇檔案,看有無已解壓縮的資料夾

!unzip -q dataset.zip 

In [ ]:
# 確定類別檔案位置和類別數

%cd /content
%cat data.yaml

In [ ]:
# 根據 data.yaml 來定義訓練的類別數
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
# 從 models 資料夾下抓取 YOLOv5 的架構
# 請注意因 YOLOv5 有4個版本,分別是 yolov5s, yolov5m, yolov5l, yolov5x, 請自行修改所需要的架構
# 並打印出 YOLOv5 的架構

%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
# 導入 IPython 套件將方便寫入代碼
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
# YOLOv5 的架構,並可從下方代碼中改寫自己所想要的架構或參數,後寫入 yolov5.yaml
# 請注意因 YOLOv5 有4個版本,分別是 yolov5s, yolov5m, yolov5l, yolov5x, 請自行修改所需要的架構
# 若不需要修改,請保持官方設定

%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # 類別數
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# 以自己所需求的 epochs 數量去訓練 YOLOv5, 可自行修改所需要的 epochs數量,這裡是 1000 epochs
# 參數說明: img(輸入的圖片尺寸), batch( batch 大小), epochs( epochs 數量), data(yaml 的路徑),cfg (模型的架構),
#           weights(權重,可修改成自己已訓練的權重路徑), name(結果的名稱), nosave(只存最後的 checkpoint), cache(內存與圖像高速緩存)

%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 1000 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

In [ ]:
# 使用 tensorboard 可視化套件
# 讀取 runs 資料夾下的結果

%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# 導入繪圖與顯示套件
from utils.plots import plot_results

# 圖片路徑(如果不對記得修改), 繪圖參數
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)

In [ ]:
# 顯示正確 Test 照片和其標註的 label
# 圖片路徑(如果不對記得修改), 繪圖參數

print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results2/test_batch2_labels.jpg', width=900)

In [ ]:
# 顯示 augment 後 test 照片和其標註label
# 圖片路徑(如果不對記得修改), 繪圖參數

print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

In [ ]:
# 已訓練完的權重會存在 runs 資料夾中

%ls runs/

In [ ]:
%ls runs/train/yolov5s_results/weights

In [ ]:
# 進行 inference, 存放test資料夾下的照片
# 使用訓練時最好的權重, best.pt, (需要注意路徑是否正確)

%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source ../test/images

In [ ]:
#顯示出 inference 後圖片, (需要注意路徑是否正確)

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'):
    display(Image(filename=imageName))
    print("\n")

In [ ]:
#下載權重檔案到 google_drive 中

from google.colab import files
files.download('./runs/train/yolov5s_results/weights/best.pt')